## Deliverable 2. Create a Customer Travel Destinations Map.

In [106]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [107]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Najran,SA,17.4924,44.1277,65.05,16,1,1.90,clear sky
1,1,New Norfolk,AU,-42.7826,147.0587,52.66,80,8,2.53,clear sky
2,2,Conceicao Do Araguaia,BR,-8.2578,-49.2647,83.79,56,96,0.47,overcast clouds
3,3,Mangrol,IN,21.1167,70.1167,78.66,64,4,5.01,clear sky
4,4,Bambous Virieux,MU,-20.3428,57.7575,73.22,77,19,12.66,light rain


In [108]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 35


In [109]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Barrow,US,71.2906,-156.7887,24.82,86,90,8.05,mist
18,18,Iqaluit,CA,63.7506,-68.5145,31.73,80,75,4.00,broken clouds
27,27,Qaanaaq,GL,77.4840,-69.3632,18.79,94,100,14.14,light snow
29,29,College,US,64.8569,-147.8028,31.95,91,90,0.00,mist
33,33,Strezhevoy,RU,60.7333,77.5889,24.78,85,99,20.74,overcast clouds
34,34,Kharp,RU,66.8014,65.8081,15.44,89,100,8.52,overcast clouds
37,37,Pangnirtung,CA,66.1451,-65.7125,30.20,80,75,13.80,broken clouds
58,58,Ilulissat,GL,69.2167,-51.1000,10.42,61,22,2.30,few clouds
64,64,Tiksi,RU,71.6872,128.8694,13.73,95,100,1.68,overcast clouds
81,81,Upernavik,GL,72.7868,-56.1549,15.26,49,22,5.57,few clouds


In [110]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 10 to 685
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_id              81 non-null     int64  
 1   City                 81 non-null     object 
 2   Country              81 non-null     object 
 3   Lat                  81 non-null     float64
 4   Lng                  81 non-null     float64
 5   Max Temp             81 non-null     float64
 6   Humidity             81 non-null     int64  
 7   Cloudiness           81 non-null     int64  
 8   Wind Speed           81 non-null     float64
 9   Current Description  81 non-null     object 
dtypes: float64(4), int64(3), object(3)
memory usage: 7.0+ KB


In [79]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#clean_df=preferred_cities_df.dropna()

In [111]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name" and rename column current description to current weather.
hotel_df.rename(columns= {'Current Description': 'Current Weather'}, inplace=True)
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
10,Barrow,US,24.82,mist,71.2906,-156.7887,
18,Iqaluit,CA,31.73,broken clouds,63.7506,-68.5145,
27,Qaanaaq,GL,18.79,light snow,77.4840,-69.3632,
29,College,US,31.95,mist,64.8569,-147.8028,
33,Strezhevoy,RU,24.78,overcast clouds,60.7333,77.5889,
34,Kharp,RU,15.44,overcast clouds,66.8014,65.8081,
37,Pangnirtung,CA,30.20,broken clouds,66.1451,-65.7125,
58,Ilulissat,GL,10.42,few clouds,69.2167,-51.1000,
64,Tiksi,RU,13.73,overcast clouds,71.6872,128.8694,
81,Upernavik,GL,15.26,few clouds,72.7868,-56.1549,


In [112]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [113]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.head(10)
hotel_df.count()

City               81
Country            81
Max Temp           81
Current Weather    81
Lat                81
Lng                81
Hotel Name         81
dtype: int64

In [114]:
# 8a. Create the output File (CSV)
output_data_file= "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [115]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp}°F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [116]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))